In [37]:
import pandas as pd
from numpy import *
from pandas import *

In [38]:
df=pd.read_csv("training-data.csv")

In [39]:
df

,Age,Salary,income,Class
0,65,65,23.4,G
1,47,15,36.5,B
2,34,75,77.2,G
3,97,40,25.4,B
4,36,100,99.0,G
5,24,60,26.0,G
6,43,56,43.2,B
7,23,90,22.6,B
8,12,23,23.2,G


In [1]:
from collections import Counter
import pandas as pd
from graphviz import Digraph


def gini_single(c: dict) -> float:
    ret = 1
    s = sum(c)
    ret -= sum(map(lambda x: (x/s)**2, c))
    return ret


def gini_double(p1: list, p2: list) -> float:
    return (sum(p1)*gini_single(p1)+sum(p2)*gini_single(p2))/(sum(p1)+sum(p2))


class node:
    attrs = []
    classList = []
    nodes = []

    def __init__(self, label="N", left=None, right=None) -> None:
        self.label = label
        self.left = left
        self.right = right
        self.spltattr = ""
        self.spltval = -1
        self.classVal = ""
        self.gini = 0
        node.nodes.append(self)

    def __repr__(self) -> str:
        # if(self.classVal == ""):
        #     return f"({self.label},{self.spltattr},{self.spltval})"
        # else:
        #     return f"({self.label},{self.classVal})"
        if(self.classVal == ""):
            return f"{self.spltattr}<={self.spltval},{self.gini}"
        else:
            return f"{self.classVal}"

    def UpdateLables(self):
        tmpHist = Counter()
        for c in self.classList:
            if(c.node == self):
                tmpHist[c.label] += 1
        if(gini_single(tmpHist.values()) == 0):
            self.classVal = list(tmpHist.keys())[0]
            return
        spltattr, splitLoc,gini = self.EvaluateSplit()
        v: Attritem
        spltval = self.attrs[spltattr][splitLoc]
        spltvalnxt = self.attrs[spltattr][splitLoc+1]
        self.spltattr = spltattr
        self.spltval = (spltval.val+spltvalnxt.val)/2
        self.left = node(label=self.label+"L")
        self.right = node(label=self.label+"R")
        self.gini=gini
        for v in self.attrs[spltattr]:
            l = self.classList[v.index]
            if(l.node == self):
                if(v.val <= spltval.val):
                    l.node = self.left
                else:
                    l.node = self.right

        self.left.UpdateLables()
        self.right.UpdateLables()

    def EvaluateSplit(self):
        min_gini = float('inf')
        # bestHist = None
        bestSplitAttr = ""
        bestSplitLoc = -1
        for a in node.attrs:
            for i, v in enumerate(node.attrs[a]):
                histL = Counter()
                histR = Counter()
                for vs in node.attrs[a]:
                    l: Classitem = node.classList[vs.index]
                    if(l.node != self):
                        continue
                    if(vs.val <= v.val):
                        histL[l.label] += 1
                    else:
                        histR[l.label] += 1
                val = gini_double(histL.values(), histR.values())
                if(val < min_gini):
                    min_gini = val
                    # bestHist = [histL, histR]
                    bestSplitAttr = a
                    bestSplitLoc = i
        return bestSplitAttr, bestSplitLoc,min_gini